<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/managed/vectaraDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Vectara托管索引
在这个笔记本中，我们将展示如何将 [Vectara](https://vectara.com) 与 LlamaIndex 结合使用。

Vectara提供了用于检索增强生成（RAG）的端到端托管服务，其中包括：
1. 从文档文件中提取文本并将其分块成句子的方法。
2. 最先进的 [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) 嵌入模型。每个文本块都使用Boomerang编码为向量嵌入，并存储在Vectara内部向量存储中。因此，当将Vectara与LlamaIndex一起使用时，您无需调用单独的嵌入模型 - 这在Vectara后端内部自动完成。
3. 一个查询服务，自动将查询编码为嵌入，并检索最相关的文本段落（包括对 [混合搜索](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) 和 [MMR](https://vectara.com/get-diverse-results-and-comprehensive-summaries-with-vectaras-mmr-reranker/) 的支持）
4. 一个选项，可以基于检索到的文档创建 [生成摘要](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview)，包括引用。

请参阅 [Vectara API文档](https://docs.vectara.com/docs/) 以获取有关如何使用API的更多信息。


## 开始

如果你在colab上打开这个笔记本，你可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index lama-index-indices-managed-vectara

要开始使用Vectara，请[注册](https://vectara.com/integrations/llamaindex)（如果尚未注册），并按照我们的[快速入门指南](https://docs.vectara.com/docs/quickstart)创建语料库和API密钥。

一旦您拥有这些内容，您可以将它们作为环境变量提供，稍后LlamaIndex代码将使用它们。

```python
import os
os.environ['VECTARA_API_KEY'] = "<YOUR_VECTARA_API_KEY>"
os.environ['VECTARA_CORPUS_ID'] = "<YOUR_VECTARA_CORPUS_ID>"
os.environ['VECTARA_CUSTOMER_ID'] = "<YOUR_VECTARA_CUSTOMER_ID>"
```


## 使用LlamaIndex和Vectara进行RAG

有几种方法可以将数据索引到Vectara中，包括：
1. 使用`VectaraIndex`的`from_documents()`或`insert_file()`方法
2. 直接在[Vectara控制台](https://console.vectara.com/)中上传文件
3. 使用Vectara的FILE_UPLOAD或标准索引API
4. 使用[vectara-ingest](https://github.com/vectara/vectara-ingest)，这是一个开源的爬虫/索引器项目
5. 使用我们的数据摄取集成合作伙伴，如Airbyte、Unstructured或DataVolo。

为此，我们将使用一组简单的小型文档，因此直接使用`VectaraIndex`进行摄取就足够了。

让我们将“AI权利法案”文档摄取到我们的新语料库中。


In [ ]:
from llama_index.indices.managed.vectara import VectaraIndex
import requests

url = "https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf"
response = requests.get(url)
local_path = "ai-bill-of-rights.pdf"
with open(local_path, "wb") as file:
    file.write(response.content)

index = VectaraIndex()
index.insert_file(
    local_path, metadata={"name": "AI bill of rights", "year": 2022}
)

### 使用Vectara查询引擎运行单个查询
现在我们已经上传了文档（或者之前已经上传了文档），我们可以直接在LlamaIndex中提出问题。这将激活Vectara的RAG管道。

要使用Vectara内部的LLM进行摘要生成，请确保在生成查询引擎时指定`summary_enabled=True`。以下是一个示例：


In [ ]:
questions = [
    "What are the risks of AI?",
    "What should we do to prevent bad actors from using AI?",
    "What are the benefits?",
]

In [ ]:
qe = index.as_query_engine(summary_enabled=True)
qe.query(questions[0]).response

"The risks associated with AI include potential biases leading to discriminatory outcomes, lack of transparency in decision-making processes, and challenges in establishing public trust and understanding of algorithmic systems [1]. Safety and efficacy concerns arise in the context of complex technologies like AI, necessitating strong regulations and proactive risk mitigation strategies [2]. The process of identifying and addressing risks before and during the deployment of automated systems is crucial to prevent harm to individuals' rights, opportunities, and access [5]. Furthermore, the impact of AI risks can be most visible at the community level, emphasizing the importance of considering and mitigating harms to various communities [6]. Efforts are being made to translate principles into practice through laws, policies, and technical approaches to ensure AI systems are lawful, respectful, accurate, safe, understandable, responsible, and accountable [7]."

如果希望以流模式返回响应，只需设置`streaming=True`


In [ ]:
qe = index.as_query_engine(summary_enabled=True, streaming=True)
response = qe.query(questions[0])

for chunk in response.response_gen:
    print(chunk.delta or "", end="", flush=True)

The risks of AI include biased data leading to discriminatory outcomes, opaque decision-making processes, and lack of public trust and understanding in algorithmic systems [1]. Organizations are implementing innovative solutions like risk assessments, auditing mechanisms, and ongoing monitoring to mitigate safety and efficacy risks of AI systems [2]. Stakeholder engagement and a risk management framework by institutions like NIST aim to address risks to individuals, organizations, and society posed by AI technology [3]. Risk identification, mitigation, and focusing on safety and effectiveness of AI systems are crucial before and during deployment to protect people’s rights, opportunities, and access [5]. The concept of communities is integral in understanding the impact of AI and automated systems, as the potential harm may be most visible at the community level [6]. Practical implementation of principles such as lawful, purposeful, accurate, safe, and accountable AI is essential to ad

### 使用Vectara聊天

Vectara还支持简单的聊天模式。在这种模式下，聊天历史由Vectara维护，因此您不必担心它。要使用它，只需调用`as_chat_engine`。

（聊天模式始终使用Vectara的摘要功能，因此您无需像以前那样显式地指定`summary_enabled=True`）


In [ ]:
ce = index.as_chat_engine()

In [ ]:
for q in questions:
    print(f"Question: {q}\n")
    response = ce.chat(q).response
    print(f"Response: {response}\n")

Question: What are the risks of AI?

Response: The risks of AI involve potential biases, opaque decision-making processes, and lack of public trust due to discriminatory outcomes and biased data [1]. To mitigate these risks, industry is implementing innovative solutions like risk assessments and monitoring mechanisms [2]. Stakeholder engagement and the development of a risk management framework by organizations like the National Institute of Standards and Technology aim to manage risks posed by AI to individuals, organizations, and society [3]. Identification and mitigation of potential risks, impact assessments, and balancing high impact risks with appropriate mitigation are crucial before and during the deployment of AI systems [5]. The Blueprint for an AI Bill of Rights emphasizes the protection of individuals from unsafe or ineffective AI systems [7].

Question: What should we do to prevent bad actors from using AI?

Response: To prevent the misuse of AI by malicious entities, seve

当然，流式传输在聊天中也适用：


In [ ]:
ce = index.as_chat_engine(streaming=True)

In [ ]:
response = ce.stream_chat("Will robots kill us all?")
for chunk in response.chat_stream:
    print(chunk.delta or "", end="", flush=True)

The search results indicate a focus on the relationship between humans and robots, emphasizing the need for co-intelligence and the best use of automated systems [2]. The discussions revolve around ensuring that automated systems are designed, tested, and protected to prevent potential harmful outcomes [1]. While there are concerns about the use of surveillance technology by companies like Amazon and Walmart, the emphasis is on balancing equities and maintaining oversight in law enforcement activities [5]. The search results do not directly answer whether robots will kill us all, but they highlight the importance of proactive protections, context-specific guidance, and existing policies to govern the use of automated systems in various settings [6].

### 代理 RAG

让我们使用 LlamaIndex 创建一个 ReAct 代理，该代理利用 Vectara 作为其 RAG 工具。
为此，您需要使用另一个 LLM 作为代理推理的驱动程序，这里我们以 OpenAI 的 GPT4o 作为示例。
（为使此工作，请确保您的环境中定义了 `OPENAI_API_KEY`）。


In [ ]:
from llama_index.core.agent import ReActAgentfrom llama_index.llms.openai import OpenAIfrom llama_index.core.tools import QueryEngineTool, ToolMetadatallm = OpenAI(model="gpt-4o", temperature=0)vectara_tool = QueryEngineTool(    query_engine=index.as_query_engine(        summary_enabled=True,        summary_num_results=5,        summary_response_lang="en",        summary_prompt_name="vectara-summary-ext-24-05-large",        reranker="mmr",        rerank_k=50,        mmr_diversity_bias=0.2,    ),    metadata=ToolMetadata(        name="Vectara",        description="Vectara Query Engine that is able to answer Questions about AI regulation.",    ),)agent = ReActAgent.from_tools(    tools=[vectara_tool],    llm=llm,    context="""        你是一个乐于助人的聊天机器人，可以使用Vectara工具回答关于AI监管的任何用户问题。        你将复杂的问题分解成更简单的问题。        你使用Vectara查询引擎来帮助回答更简单的问题。    """,    verbose=True,)

In [ ]:
问题 = """    人工智能的风险是什么？有哪些好处？    从专家的观点比较和对比，并提供赞成和反对的论点的摘要。"""print(agent.chat(question).response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: Vectara
Action Input: {'input': 'What are the risks of AI?'}
Observation: The risks of AI include biased data leading to discriminatory outcomes, opaque decision-making processes, and a lack of public trust and understanding in algorithmic systems. Mitigation strategies discussed involve ongoing transparency, participatory design, and engaging with impacted communities to understand potential harms and integrate protections into the design of AI systems [1]. Additionally, there's a focus on identifying and mitigating risks before deployment, particularly those impacting people's rights, opportunities, or safety, with a strong emphasis on avoiding systems that inherently violate safety norms [5].
Thought: I have obtained information about the risks of AI. Now, I need to gather information about the benefits of AI to provide a comprehensive comparison.
Action: Vectara
Actio